In [1]:
import ee

from naip_cnn.inference import NAIPTFRecord

ee.Initialize()

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [6]:
BBOX = ee.Geometry.Polygon(
    [
        [
            [-119.37239004502233, 44.48079613290612],
            [-118.57725454697545, 44.48079613290612],
            [-118.57725454697545, 44.81785572318615],
            [-119.37239004502233, 44.81785572318615],
        ]
    ]
)

naip = NAIPTFRecord(id="MAL", footprint=(150, 150), res=1, year=2020, bounds=BBOX)

In [7]:
task = naip.export_to_drive()